In [ ]:
# very powerful, but discarded because requires at least 40gb cards to run and may be harder to tweak than diffusers
!git clone https://github.com/xdit-project/xDiT.git
%cd xDiT
%pip install flask
%pip install -e ".[diffusers,flash-attn]"
!python3 ./http-service/launch_host.py --config ./http-service/config.json
%cd /workspace/dtback

In [ ]:
# ComfyUI setup
%cd  /workspace/dtback
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /workspace/dtback/ComfyUI
# !pip install -r requirements.txt
%cd /workspace/dtback/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git

!huggingface-cli download black-forest-labs/FLUX.1-Fill-dev flux1-fill-dev.safetensors --local-dir /workspace/dtback/ComfyUI/models/unet

!huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir /workspace/dtback/ComfyUI/models/vae
!huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir /workspace/dtback/ComfyUI/models/clip
!huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir /workspace/dtback/ComfyUI/models/clip

!huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir /workspace/dtback/ComfyUI/models/loras

!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
import base64
t = 'Mk9WSkJ6c2p4QzJzcVZLdEFJNjBWdndhWmhEXzd1UUMzVm8xZ3dMVmllS0JOcHdvNg=='
k = base64.b64decode(t.encode()).decode()
!ngrok config add-authtoken {k}
# ngrok http 8000

%cd /workspace/dtback
!python3 ComfyUI/main.py --port 8000

In [ ]:
# img2img amine
%cd ~
import time
start = time.time()
output = flux.img_to_img(
    prompt="""
    
    anime picture of two boys standing in a room near a table with pizza

    """,
    image="img2.jpg",
    strength=0.63,
    sample_steps=20,
    width=1024,
    height=768,
    seed=1000,
    cfg_scale=5,
    style_strength=20,
    sample_method="euler",
)
delta = round(time.time() - start)
output[0].convert('RGB').save("output_img.jpg")
from IPython.display import clear_output
clear_output()
print(f'took {delta} seconds')
output[0].show()